# Data model

Compared to what is provided in most general-purpose programming languages, only a small set of abstract data types are needed for analysis (or high-level programming in general). Numbers, booleans, and fixed-size (rectangular) arrays of numbers and booleans are sufficient for data analysis in most fields of study. Particle physicists demonstrably need variable-length lists of numbers and booleans as well: interest in awkward-array has overwhelmingly focused on the `JaggedArray` class, which only provides this capability. For the intuitive notion of a "particle object," some sort of record type is needed as well. To manipulate sets without restriction, we also need heterogeneity, which can be expressed by a union type.

## Data types

The following four type generators ("PLUR") provide a system of surprising generality:

   * **P**rimitive integers, floating-point numbers, booleans, and any fixed byte-width value (e.g. UUIDs, IP addresses, ...),
   * **L**ists of variable length but homogeneous type,
   * **U**nions of heterogeneous types, such as "electrons and muons" (with different fields in each), and
   * **R**ecords of named, typed fields (a.k.a. objects, structs, composites, classes...).

For instance, JSON (with a schema) is a PLUR system, in which numbers, boolean, and `null` are the primitives, and strings are regarded as a special case of "lists of 1-byte characters." Protobuf, Avro, Thrift, Parquet, and Arrow are statically typed PLUR systems. Unions and records are the sum types and product types of [algebraic type theory](https://en.wikipedia.org/wiki/Algebraic_data_type), respectively. Only one thing from general-purpose programming might be missed by physicists:

   * **P**ointers between objects.

However, we can add this as a fifth type generator ("PLURP") by allowing cross-references and circular references. In a PLUR system, data structures are trees with a maximum depth, limited by the type schema, but in a PLURP system, data structures may be arbitrarily deep trees or even graphs. Awkward-array is PLURP with extra features beyond just representing types.

Data types in a general-purpose programming language can be constructed from the above if interpreted through the appropriate interfaces. For instance, an open file object is an integer that makes system calls, a linked list is a tree of records, and a hash-table is a list with hash-collision handling. PLURP provides a layer of abstraction between raw, serialized memory (e.g. the arrays and structs of the C programming language) and rarified types of a high-level language (e.g. classes with hidden implementations).

## Multi-paradigm columnar processing

Awkward-array has been useful as a Numpy extension for particle physics, and I expect its role to increase. However, I don't think the array-programming paradigm is good for all problems. In fact, I'd like to provide three ways to perform computations on these same data structures:

   * array programming, in which the columnar nature of the arrays is visible and there is no index,
   * imperative programming in Numba, in which the columnar nature of the arrays is hidden—the user works with "lists" and "records" in compiled Python—and there is no index, and
   * declarative programming, in which the columnar nature of the arrays is hidden and there is an index to define identity for set operations.

My goal is for the three paradigms to be usable on the same data structures without translation. For example, a physicist-user might apply a first transformation of their data as an array, then do something more complex in a Numba-compiled block, treating the records as Python objects (though Numba compiles their actions into array manipulations under the hood), then do something even more complex, relying on the identity of records in a way not possible in imperative programming, using a declarative language like PartiQL.

## Mini-awkward-array

Rather than using awkward-array in this demo, I reimplemented a simple version of it so that relationships between the types and the index is more clear.

This implementation has only four classes: `PrimitiveArray`, `ListArray`, `UnionArray`, and `RecordArray`. As in awkward-array, the data are stored in columns but may be thought of as nested objects.

In [1]:
import data

events = data.RecordArray({
        "muons": data.ListArray([0, 3, 3, 5], [3, 3, 5, 9], data.RecordArray({
            "pt": data.PrimitiveArray([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]),
            "iso": data.PrimitiveArray([0, 0, 100, 50, 30, 1, 2, 3, 4])
    })),
        "jets": data.ListArray([0, 5, 6, 8], [5, 6, 8, 12], data.RecordArray({
            "pt": data.PrimitiveArray([1, 2, 3, 4, 5, 100, 30, 50, 1, 2, 3, 4]),
            "mass": data.PrimitiveArray([10, 10, 10, 10, 10, 5, 15, 15, 9, 8, 7, 6])
        })),
        "met": data.PrimitiveArray([100, 200, 300, 400])
    })

The semi-realistic event structure above is defined in its columnar representation—Python lists are stand-ins for the arrays we would use in an efficient system. We can extract data as jagged arrays:

In [2]:
events["muons"]["pt"]

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5], [6.6, 7.7, 8.8, 9.9]]>

Or as the nested objects the arrays represent.

In [3]:
events[0]["muons"].tolist()

[{'pt': 1.1, 'iso': 0}, {'pt': 2.2, 'iso': 0}, {'pt': 3.3, 'iso': 100}]

In awkward-array, we would describe the high-level type of this structure as follows:

```
[0, 4) -> "muons" -> [0, ?) -> "pt" -> float64
                               "iso" -> float64
          "jets" -> [0, ?) -> "pt" -> float64
                              "mass" -> float64
          "met" -> float64
```

because an array is like a function that takes an integer or string in square brackets and returns something else—an array or a scalar primitive. The first argument after `events` can be any non-negative integer less than `4`, and this returns an array/function that takes `"muons"`, `"jets"`, or `"met"`. The return type of the next argument depends on which string was passed. The variable-length lists take a non-negative integer `[0, ?)` because its limits are too complex to encode in the type description. (The size of the type description should not be allowed to scale with the size of the array, and this limits its expressiveness for variable-length lists.) If awkward-array is based on Numpy, the leaves of its type terminate on Numpy dtypes, such as `float64`. If the array includes a cross-reference or circular reference, the type description would be a graph with interconnections, not a tree.

In awkward-array, it's possible to change the order of these arguments:

In [4]:
# [0, 4) before "muons"/"jets"/"met"
events[0]["muons"].tolist()

[{'pt': 1.1, 'iso': 0}, {'pt': 2.2, 'iso': 0}, {'pt': 3.3, 'iso': 100}]

In [5]:
# "muons"/"jets"/"met" before [0, 4)
events["muons"][0].tolist()

[{'pt': 1.1, 'iso': 0}, {'pt': 2.2, 'iso': 0}, {'pt': 3.3, 'iso': 100}]

Passing a string argument to a `ListArray`, as though it were a `RecordArray`, creates a `ListArray` of one of the nested `RecordArray`'s fields. It is a projection through the nested records.

In [6]:
events["muons"]["pt"]

<Array [[1.1, 2.2, 3.3], [], [4.4, 5.5], [6.6, 7.7, 8.8, 9.9]]>

In [7]:
events["muons"]["iso"]

<Array [[0, 0, 100], [], [50, 30], [1, 2, 3, 4]]>

Formally, we find that string arguments can commute leftward through integer arguments, though string arguments do not commute with string arguments (you can't reverse the order of nested records) and integer arguments do not commute with integer arguments (you can't reverse the order of nested lists). A string argument cannot commute to the right in its placement in the data type: you can't say

In [8]:
events[0][2]["muons"]["pt"]

KeyError: 2

because the three options `"muons"`, `"jets"`, `"met"` do not lead to the same types: `"met"` yields a scalar `float64`, which cannot take any array arguments. Because of this technicality, we have to say, "string indexes commute with integer indexes *up to their rightmost position,*" where the "rightmost position" is the position of the string in the data type.

Notwithstanding that technicality, the commutation relation is an important feature and will be used to define awkward indexes.

## Indexes and keys

The concept that an SQL-like query language would add to awkward-array is indexing—giving each element of the arrays (at all levels) a unique identifier ("key").

Just as the data types are determined by what types of arguments can be placed in square brackets and what types that returns, the index is determined by a specific sequence of arguments that lead to a given data element. For example:

In [9]:
events[0]["muons"][2]["pt"]

3.3

The sequence `0`, `"muons"`, `2`, `"pt"` leads to 3.3, so this value of 3.3 could be indexed by `[0, "muons", 2, "pt"]`. There may be other values of 3.3 in the dataset, but they are different entities with different index keys.

In relational databases, this is called a [surrogate key](https://en.wikipedia.org/wiki/Surrogate_key), a declaration that this entity is unique by fiat, as opposed to a [natural key](https://en.wikipedia.org/wiki/Natural_key) that determines uniqueness through the value of the measured data. For instance, if we had decided that all muons with a pT of 3.3 are the same muon, those would be natural keys. Natural keys are meaningful for time measurements in time series or latitude/longitude coordinates in geographical data, but no values in a particle physics dataset would make sense as a natural key. Even if all low-level signals in a particle physics collision were perfectly repeated by a second collision, we would want to consider that second instance as a distinct event, and all of the particles it contains as distinct from the particles of the first event. Thus, we generate surrogate keys based on their location in the array structure.

Because of the commutivity of integers and strings, the following are equivalent:

```
         [0, "muons", 2, "pt"]
         [0, "muons", "pt", 2]
         ["muons", 0, 2, "pt"]
         ["muons", 0, "pt", 2]
         ["muons", "pt", 0, 2]
(but not [0, 2, "muons", "pt"])
```

Rather than choosing one arbitrarily, we can express this commutivity by separating the index into a row index and a column index:

```
row=[0, 2], col=["muons", "pt"]
```

Note that we can only perform this assignment in data without cross-references—PLUR, not PLURP. To accomodate indexing, future versions of awkward-array will need to demote cross-references to a second-class status: a non-cross-referenced structure may be built from direct references, but interconnections would have to be called out as "soft links" or "weak references" so that they can be ignored while assigning indexes. (Alternatively, we could do a depth-first walk with breadcrumbs to avoid walking over the same structure twice, but indexes derived that way would depend on the order in which record fields are walked. It would be safer to call out "direct references" from "cross-references" explicitly.)

### Row and column indexes in Pandas

This is the principle that Pandas uses to express arbitrary lists of records as a two-dimensional table. To see this in a few examples, install awkward-array and Pandas:

In [10]:
!pip install awkward pandas

In [11]:
import awkward
awkward.topandas(awkward.fromiter(events)["muons"], flatten=True)

iso   pt
0 0    0  1.1
  1    0  2.2
  2  100  3.3
2 0   50  4.4
  1   30  5.5
3 0    1  6.6
  1    2  7.7
  2    3  8.8
  3    4  9.9

In [12]:
awkward.topandas(awkward.fromiter(events)["jets"], flatten=True)

mass   pt
0 0   10    1
  1   10    2
  2   10    3
  3   10    4
  4   10    5
1 0    5  100
2 0   15   30
  1   15   50
3 0    9    1
  1    8    2
  2    7    3
  3    6    4

In [13]:
# deeply nested records → nested column keys
awkward.topandas(awkward.fromiter([
    {"left": {"x": 1, "y": {"z": 1}}, "right": {"a": 1, "b": 1, "c": 1}}, 
    {"left": {"x": 2, "y": {"z": 2}}, "right": {"a": 2, "b": 2, "c": 2}}, 
    {"left": {"x": 3, "y": {"z": 3}}, "right": {"a": 3, "b": 3, "c": 3}}
]), flatten=True)

left    right      
     x  y     a  b  c
        z            
0    1  1     1  1  1
1    2  2     2  2  2
2    3  3     3  3  3

In [14]:
# deeply nested lists → nested row keys
awkward.topandas(awkward.fromiter([
    [[{"x": 100}, {"x": 100}], [{"x": 100}, {"x": 100}, {"x": 100}]],
    [],
    [[{"x": 300}], [], [{"x": 300}, {"x": 300}], [{"x": 300}]]
]), flatten=True)

x
0 0 0  100
    1  100
  1 0  100
    1  100
    2  100
2 0 0  300
  2 0  300
    1  300
  3 0  300

In Pandas, rows and columns can be assigned with a `MultiIndex`. Whenever Pandas `DataFrames` are merged, rows and columns with matching `MultiIndex` tuples are considered equivalent and matched, even if they are out of order. We will do the same with data in PartiQL, which frees it from list order and possible duplication, acting at the level of sets.

### Index visibility

Since surrogate keys are artificial, there are two schools of thought on their visibility:

   * Hall, Owlett, and Todd (1976) describe surrogate keys as an integer field in a table;
   * Wieringa and De Jonge (1991) describe them as an internal implementation detail.

If surrogate keys are visible, users can express sampling without replacement through an inequality on the surrogate key:

```sql
... FROM muons m1 JOIN muons m2 ON m1.id < m2.id
```

If they are not visible, constructs like this have to be provided by the language—they can't be built manually. For PartiQL, I have chosen to hide them as an implementation detail and provide syntactic constructs for sampling without replacement:

```
muons as (m1, m2)
```

and sampling with replacement:

```
muons as m1 cross muons as m2
```

This puts more separation between *what* a user wants and *how* it is implemented. (Sampling without replacement in SQL could use `m1.id < m2.id`, `m1.id > m2.id`, `(m1 + n) % COUNT(muons) < (m2 + n) % COUNT(muons)`, or `permutation(m1.id) < permutation(m2.id)` for any permutation. Detecting the intent can be hard.)

### Order visibility

With hidden surrogate indexes, it would also be possible to entirely hide the order of lists. This is theoretically appealing because then the language would truly be a language of sets. Introducing a single function that reveals the order of a list would make it necessary to provide options to control the order in all other operations (such as SQL's `ORDER BY`).

For simplicity (and to see how far we can take it), PartiQL does not reveal the order of lists. A "list" in awkward-array is a "set" in PartiQL. (Order is revealed if we apply awkward-array operations *after* a PartiQL operation, but that's beyond the PartiQL language.)

### Joins and index compatibility

Many mathematical operations apply to scalars, such as addition, square root, exponentiation, etc. In array programming, they can be applied to all members of two or more arrays if the arrays have the same length (or n-dimensional shape) by lining up the arrays and performing the operation on each scalar element. Awkward-array extends this to arrays of variable-length lists, as long as the lengths in each input are the same when aligned. The same can be applied to sets if they have the same index.

Aligning two or more sets is called a "join" ("merge" in Pandas). The internal representations of the sets might have different list orders, so "alignment" for sets includes reordering.

What if a user tries to join `muons` and `jets` to add their `pt`? Such an operation should be illegal because we can never guarantee that there's the same number of `muons` and `jets` in each event and there is no natural mapping from one set to the other set. Awkward-array *usually* forbids such an operation because there is *usually* at least one event in a large dataset with a different number of `muons` than `jets`. With indexes, we can *always* forbid such an operation because they have non-overlapping indexes (no keys in common).

What if a user tries to join `filteredMuons` or `muonsWithCorrections` to `muons`? Such an operation should be legal and it would often be useful. If there is a different number of `filteredMuons` or `muonsWithCorrections` than `muons`, even in one event, awkward-array cannot combine them. With indexes, we can *always* perform such operations because the filtered or transformed muons carry over their indexes—their identities—through the filtering or transformation.

In particular, we can even perform computations in Directed Acyclic Graphs (DAGs) that change the number or order of collections. For example:

```python
B = f(A)
C = g(A)
D = h(B, C)
```

where `f` and `g` might change the number or order of values in `A`, `h` is possible because elements of `B` can be matched to unique elements of `C` (and vice-versa) through the index keys they both inherited from `A` (passed through `f` and `g`).

To ensure that `B` and `C` are both transformed versions of the same particles, keys must be compared by reference, not by value. The second muon in the first event has row index `[0, 1]` and the second jet in the first event also has row index `[0, 1]`, but they are not the same key because they don't derive from the same index. We can't disambiguate them on their column indexes, `["muons", "pt"]` and `["jets", "pt"]`, because we should be able to join different columns, such as `["muons", "pt"]` and `["muons", "iso"]`. When indexes are created, each index should have a globally unique identifier that gets passed through operations so that we can distinguish, e.g. `#0[0, 1]` from `#1[0, 1]`.

## Demonstration of awkward indexes

The demo code includes a `setindex` method that assigns index keys to all reachable elements in an awkward-array.

In [15]:
events.setindex()

In [16]:
events.contents["muons"].content.contents["pt"]

<Array [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]>

In [17]:
events.contents["muons"].content.contents["pt"].row

RowIndex([(0, 0), (0, 1), (0, 2), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (3, 3)], <Ref 1>)

Each primitive array, such as `events.contents["muons"].content.contents["pt"]`, has a `RowIndex` with a `RowKey` for every element and a `Ref` for referential identity.

If you reach down to individual elements, you'll see that the `Ref` is attached to each `RowKey`.

In [19]:
events.contents["muons"].content.contents["pt"].row[3]

RowKey((2, 0), <Ref 1>)

A different primitive array in the same object, for which indexes must be compatible, has the same `Ref`.

In [21]:
events.contents["muons"].content.contents["pt"].row

RowIndex([(0, 0), (0, 1), (0, 2), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (3, 3)], <Ref 1>)

In [20]:
events.contents["muons"].content.contents["iso"].row[3]

RowKey((2, 0), <Ref 1>)

For different objects, like muons and jets, the indexes are incompatible, even if they happened to have the same number of elements per event.

In [35]:
events.contents["jets"].content.contents["pt"].row

RowIndex([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), (3, 3)], <Ref 2>)

In [36]:
events.contents["muons"].content.contents["pt"].row.ref, events.contents["jets"].content.contents["pt"].row.ref

(<Ref 1>, <Ref 2>)

There is only one `ColIndex` associated with the whole primitive array because the whole array represents a column.

Columns do not need to be compared for referential equality because they are always compatible. (It is assumed that the user would be combining data from different fields of the same objects.)

In [22]:
events.contents["muons"].content.contents["pt"].col

ColIndex('muons', 'pt')

In fact, in the whole implementation of PartiQL, column indexes are never needed. They seem to be superflous: a non-toy implementation might not even need such a concept.

### Unreachable elements

Awkward arrays can have unreachable elements—data that are inaccessible because we wish to avoid propagating slices through all layers of the hierarchy, to make multi-step append operations effectively atomic, because TLorentzVectors from ROOT include TObject bytes that aren't relevant for analysis, or any other reason.

Here is an example:

In [23]:
example = data.ListArray([3, 999, 0], [6, 999, 2],
              data.PrimitiveArray([4.4, 5.5, 12345, 1.1, 2.2, 3.3, 98765]))
example.tolist()

[[1.1, 2.2, 3.3], [], [4.4, 5.5]]

In this example, not only are the inner list boundaries out of order, but there is no way to reach `12345` or `98765`.

These elements do not get indexes.

In [24]:
example.setindex()

In [25]:
example.content

<Array [4.4, 5.5, 12345, 1.1, 2.2, 3.3, 98765]>

In [26]:
example.content.row

RowIndex([(2, 0), (2, 1), None, (0, 0), (0, 1), (0, 2), None], <Ref 4>)

In a non-toy implementation, in which the index is an array, these values may be filled with `-1`. (Reachable indexes are all non-negative.)

### Heterogeneous data

Although datasets produced by reconstruction are rarely heterogeneous, intermediate steps in PartiQL frequently involve unions. Index keys have a sensible definition for heterogenous types.

In [29]:
egamma = data.UnionArray([0, 0, 1, 0, 1, 1, 1, 0, 0], [0, 1, 0, 2, 1, 2, 3, 3, 4], [
        data.RecordArray({
            "q": data.PrimitiveArray([1, -1, -1, 1, 1]),
            "pt": data.PrimitiveArray([10, 20, 30, 40, 50])
        }),
        data.RecordArray({
            "pt": data.PrimitiveArray([1.1, 2.2, 3.3, 4.4])
        })
    ])

In [34]:
egamma.tolist()

[{'q': 1, 'pt': 10},
 {'q': -1, 'pt': 20},
 {'pt': 1.1},
 {'q': -1, 'pt': 30},
 {'pt': 2.2},
 {'pt': 3.3},
 {'pt': 4.4},
 {'q': 1, 'pt': 40},
 {'q': 1, 'pt': 50}]

In [30]:
egamma.setindex()

In [32]:
egamma.contents[0].contents["pt"], egamma.contents[1].contents["pt"]

(<Array [10, 20, 30, 40, 50]>, <Array [1.1, 2.2, 3.3, 4.4]>)

In [33]:
egamma.contents[0].contents["pt"].row, egamma.contents[1].contents["pt"].row

(RowIndex([(0,), (1,), (3,), (7,), (8,)], <Ref 6>),
 RowIndex([(2,), (4,), (5,), (6,)], <Ref 7>))

Note that the key values for data in different branches of the union are both non-overlapping and have different references. An electron will never be confused with a photon.

## Intermediate objects

Objects at all levels of the tree have indexes, not just leaves.

In [39]:
events.contents["muons"].row

RowIndex([(0,), (1,), (2,), (3,)], <Ref 0>)

In [40]:
egamma.row

RowIndex([(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)], <Ref 5>)

## Rowwise vs columnar calculations

In a non-toy implementation, all operations should be computed on columns because it is more efficient in many ways (fewer branches in the code, less data to swap into CPU cache, and possibly even less data to load from disk). However, it is also more complicated. For simplicity in this toy implementation, the columnar data will be decomposed into rowwise objects with index keys attached.

In [41]:
data.instantiate(events)

List{ 
    Rec#0(0)(){ 
        muons = List#0(0)('muons'){ 
            Rec#1(0 0)('muons'){ 
                pt = Value#1(0 0)('muons' 'pt'){ 1.1 }
                iso = Value#1(0 0)('muons' 'iso'){ 0 }
            }
            Rec#1(0 1)('muons'){ 
                pt = Value#1(0 1)('muons' 'pt'){ 2.2 }
                iso = Value#1(0 1)('muons' 'iso'){ 0 }
            }
            Rec#1(0 2)('muons'){ 
                pt = Value#1(0 2)('muons' 'pt'){ 3.3 }
                iso = Value#1(0 2)('muons' 'iso'){ 100 }
            }
        }
        jets = List#0(0)('jets'){ 
            Rec#2(0 0)('jets'){ 
                pt = Value#2(0 0)('jets' 'pt'){ 1 }
                mass = Value#2(0 0)('jets' 'mass'){ 10 }
            }
            Rec#2(0 1)('jets'){ 
                pt = Value#2(0 1)('jets' 'pt'){ 2 }
                mass = Value#2(0 1)('jets' 'mass'){ 10 }
            }
            Rec#2(0 2)('jets'){ 
                pt = Value#2(0 2)('jets' 'pt'){ 3 }
                mass 

In [42]:
data.instantiate(egamma)

List{ 
    Rec#6(0)(){ 
        q = Value#6(0)('q'){ 1 }
        pt = Value#6(0)('pt'){ 10 }
    }
    Rec#6(1)(){ 
        q = Value#6(1)('q'){ -1 }
        pt = Value#6(1)('pt'){ 20 }
    }
    Rec#7(2)(){ 
        pt = Value#7(2)('pt'){ 1.1 }
    }
    Rec#6(3)(){ 
        q = Value#6(3)('q'){ -1 }
        pt = Value#6(3)('pt'){ 30 }
    }
    Rec#7(4)(){ 
        pt = Value#7(4)('pt'){ 2.2 }
    }
    Rec#7(5)(){ 
        pt = Value#7(5)('pt'){ 3.3 }
    }
    Rec#7(6)(){ 
        pt = Value#7(6)('pt'){ 4.4 }
    }
    Rec#6(7)(){ 
        q = Value#6(7)('q'){ 1 }
        pt = Value#6(7)('pt'){ 40 }
    }
    Rec#6(8)(){ 
        q = Value#6(8)('q'){ 1 }
        pt = Value#6(8)('pt'){ 50 }
    }
}

It should be clear, however, that all operations implemented on rowwise objects can be translated to columnar data for faster processing.

# Implications for awkward 1.0

Awkward-array will soon be rewritten for a variety of reasons, and that gives us opportunity to revisit fundamental questions. Knowing now what is required for set operations—a `d×N` matrix of integers where `d` is the depth and `N` is the length of an internal array, with no need for a column index—awkward 1.0 can be written to accomodate for this data structure. In fact, if an index is present, it can be passed through the array programming interface, to ensure that array programming and declarative set operations can be interleaved.